In [59]:
import pandas as pd
import numpy as np

In [60]:
df_test = pd.read_csv('../data/processed/test/test_processed.csv')
df_test.head()

,metal_linker,organic_linker1,organic_linker2,topology,Co2N2_selectivity,heat_adsorption,functional_groups1,functional_groups2
0,2,5,26,5,36.639791,7.005640,9,20
1,4,6,17,0,18.390691,5.119399,20,18
2,3,11,17,5,13.062850,5.045400,16,5
3,2,1,28,5,9.601198,5.106238,1,20
4,3,8,19,5,12.974954,5.287639,0,14


In [61]:
map_old_new_colname = {
    'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]': 'heat_adsorption',
    'CO2/N2_selectivity': 'Co2N2_selectivity',
    'CO2_working_capacity [mL/g]': 'CO2_working_capacity'
}

In [62]:
df_test = df_test.rename(columns=map_old_new_colname)

In [63]:
test_id = [i for i in range(68614, 85613+1)]

# LGBM

In [64]:
from lightgbm import Booster

In [65]:
model_filename = input('Enter LGBM model filename: ')

lgbm_model = Booster(model_file=f'../models/lgbm/{model_filename}.txt')

In [66]:
lgbm_prediction = lgbm_model.predict(df_test)

In [67]:
df_lgbm = pd.DataFrame({'id': test_id, 'CO2_working_capacity [mL/g]': lgbm_prediction})

In [68]:
df_lgbm.head()

,id,CO2_working_capacity [mL/g]
0,68614,212.241472
1,68615,87.477139
2,68616,62.876545
3,68617,65.304226
4,68618,65.188592


In [69]:
output_filename = input('Enter lgbm output filename: ')

df_lgbm.to_csv(f'../models/lgbm/output/csv/{output_filename}.csv', index=False)

# CatBoost

In [70]:
from catboost import CatBoostRegressor

In [71]:
model_filename = input('Enter CatBoost model filename: ')

catboost_model = CatBoostRegressor().load_model(f'../models/catboost/{model_filename}.json', format='json')

In [72]:
catboost_prediction = catboost_model.predict(df_test)

In [73]:
df_cat = pd.DataFrame({'id': test_id, 'CO2_working_capacity [mL/g]': catboost_prediction})

In [74]:
df_cat.head()

,id,CO2_working_capacity [mL/g]
0,68614,180.110638
1,68615,82.170020
2,68616,62.434579
3,68617,63.053140
4,68618,59.400838


In [75]:
output_filename = input('Enter cat output filename: ')

df_cat.to_csv(f'../models/catboost/output/csv/{output_filename}.csv', index=False)